# SAE-98 - Nettoyage Dataset Users

**Objectif:** Charger, analyser et nettoyer le dataset Users

**Input:** `data/raw/yelp_academic_dataset_user4students.jsonl`

**Output:** `data/cleaned/users_clean.parquet`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
print("✅ Imports réussis")

In [ ]:
RAW_DATA_PATH = Path('../data/raw/yelp_academic_dataset_user4students.jsonl')
CLEANED_DATA_PATH = Path('../data/cleaned/users_clean.parquet')
CLEANED_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"📂 Chargement: {RAW_DATA_PATH}")
print(f"💾 Sauvegarde: {CLEANED_DATA_PATH}")

In [ ]:
print("⏳ Chargement... (1-2 min)")
users_df = pd.read_json(RAW_DATA_PATH, lines=True)
print(f"✅ {users_df.shape[0]:,} lignes, {users_df.shape[1]} colonnes")

In [ ]:
users_df.head()

In [ ]:
users_df.info()

In [ ]:
missing = pd.DataFrame({
    'Colonne': users_df.columns,
    'Manquantes': users_df.isnull().sum(),
    'Pct': (users_df.isnull().sum() / len(users_df) * 100).round(2)
}).sort_values('Pct', ascending=False)
missing[missing['Manquantes'] > 0]

In [ ]:
users_clean = users_df.copy()
print(f"📋 Initial: {len(users_clean):,} lignes")

In [ ]:
# Supprimer doublons
dups = users_clean.duplicated(subset=['user_id']).sum()
if dups > 0:
    users_clean = users_clean.drop_duplicates(subset=['user_id'])
    print(f"❌ {dups} doublons supprimés")
else:
    print("✅ Aucun doublon")

In [ ]:
# Supprimer lignes sans user_id
if users_clean['user_id'].isnull().any():
    before = len(users_clean)
    users_clean = users_clean.dropna(subset=['user_id'])
    print(f"❌ {before - len(users_clean)} sans user_id")
else:
    print("✅ Tous les user_id présents")

In [ ]:
# Remplir valeurs manquantes pour colonnes numériques
numeric_cols = ['review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars']
for col in numeric_cols:
    if col in users_clean.columns:
        users_clean[col] = users_clean[col].fillna(0)

# Remplir valeurs manquantes pour colonnes texte
if 'name' in users_clean.columns:
    users_clean['name'] = users_clean['name'].fillna('Unknown')

print("✅ Valeurs manquantes traitées")

In [ ]:
# Valider review_count >= 0
if 'review_count' in users_clean.columns:
    before = len(users_clean)
    users_clean = users_clean[users_clean['review_count'] >= 0]
    if before > len(users_clean):
        print(f"❌ {before - len(users_clean)} review_count négatifs")
    else:
        print("✅ review_count valides")

In [ ]:
# Valider average_stars entre 0 et 5
if 'average_stars' in users_clean.columns:
    before = len(users_clean)
    users_clean = users_clean[(users_clean['average_stars'] >= 0) & (users_clean['average_stars'] <= 5)]
    if before > len(users_clean):
        print(f"❌ {before - len(users_clean)} average_stars invalides")
    else:
        print("✅ average_stars valides (0-5)")

In [ ]:
# Convertir dates
if 'yelping_since' in users_clean.columns:
    users_clean['yelping_since'] = pd.to_datetime(users_clean['yelping_since'], errors='coerce')
    invalid = users_clean['yelping_since'].isnull().sum()
    if invalid > 0:
        print(f"⚠️ {invalid} dates invalides (conservées comme NaT)")
    else:
        print("✅ Dates converties")

In [ ]:
print("📊 RÉSUMÉ")
print("=" * 50)
print(f"Initial:    {len(users_df):,}")
print(f"Final:      {len(users_clean):,}")
print(f"Supprimé:   {len(users_df) - len(users_clean):,}")
print(f"Conservé:   {(len(users_clean) / len(users_df) * 100):.2f}%")

In [ ]:
# Distribution review_count
if 'review_count' in users_clean.columns:
    plt.figure(figsize=(12, 5))
    plt.hist(users_clean['review_count'], bins=50, edgecolor='black')
    plt.xlabel('Nombre d\'avis')
    plt.ylabel('Nombre d\'utilisateurs')
    plt.title('Distribution du Nombre d\'Avis par Utilisateur')
    plt.xlim(0, users_clean['review_count'].quantile(0.95))
    plt.show()

In [ ]:
# Distribution average_stars
if 'average_stars' in users_clean.columns:
    plt.figure(figsize=(10, 5))
    users_clean['average_stars'].hist(bins=30, edgecolor='black')
    plt.xlabel('Note moyenne')
    plt.ylabel('Nombre d\'utilisateurs')
    plt.title('Distribution des Notes Moyennes des Utilisateurs')
    plt.show()

In [ ]:
# Sauvegarder
print("⏳ Sauvegarde...")
users_clean.to_parquet(CLEANED_DATA_PATH, index=False, compression='snappy')
print(f"✅ Sauvegardé: {CLEANED_DATA_PATH}")
print(f"📊 Taille: {CLEANED_DATA_PATH.stat().st_size / 1024**2:.2f} MB")

In [ ]:
# Test rechargement
test_df = pd.read_parquet(CLEANED_DATA_PATH)
print(f"✅ Test réussi: {len(test_df):,} lignes")

## ✅ SAE-98 Terminé

**Prochaines étapes:**
- SAE-67: Dashboard Profils Reviewers
- SAE-68: Dashboard Performance Établissements